In [1]:
import pm4py

import random

import numpy as np

from datetime import datetime, timedelta

from pm4py.objects.log.importer.xes import importer as xes_importer

from pm4py.algo.filtering.log.attributes import attributes_filter

from pm4py.objects.log.exporter.xes import exporter as xes_exporter

import copy

import os 

In [2]:
def Has_Timestamp(log):
    if "time:timestamp" in log[0][0]:
        return True
    else:
        return False

In [3]:
def get_voc(log):
    activities = attributes_filter.get_attribute_values(log, "concept:name")
    return [*activities]

In [4]:
def add_start_and_end(log, name_start, name_end):
    for i in range(0, len(log)):
        dummy_start_event = copy.deepcopy(log[i][0])
        dummy_end_event = copy.deepcopy(log[i][-1])
        
        log[i].insert(0, dummy_start_event)
        log[i].append(dummy_end_event)
        
        log[i][0]['concept:name'] = name_start
        log[i][-1]['concept:name'] = name_end

        if Has_Timestamp(log) == True:
            new_start_timestamp = log[i][0]['time:timestamp'] - timedelta(seconds=1)
            new_end_timestamp = log[i][-1]['time:timestamp'] + timedelta(seconds=1)
            log[i][0]['time:timestamp'] = new_start_timestamp
            log[i][-1]['time:timestamp'] = new_end_timestamp
    return log

In [5]:
def add_one_delete(log, fraction):
    which = np.random.permutation(int(fraction*len(log)))
    for i in range(0, len(log)):
        if i in which:
            if len(log[i]) < 1:
                continue
            place = random.randint(0, len(log[i])-1)
            del log[i]._list[place]
    return log
    
def add_one_replace(log, fraction, voc):
    which = np.random.permutation(int(fraction*len(log)))
    for i in range(0, len(log)):
        if i in which:
            if len(log[i]) < 1:
                continue
            place = random.randint(0, len(log[i])-1)
            new_act = random.choice(voc)
            log[i][place]['concept:name'] = new_act
    return log
       
def add_one_addition(log, fraction, voc):
    which = np.random.permutation(int(fraction*len(log)))
    for i in range(0, len(log)):
        if i in which:
            place = random.randint(0, len(log[i])-1)
            new_act = random.choice(voc)
            dummy_event = copy.deepcopy(log[i][place])
            log[i].insert(place, dummy_event)
            log[i][place]['concept:name'] = new_act
            if Has_Timestamp(log) == True:
                old_timestamp = copy.deepcopy(log[i][place]['time:timestamp'])
                new_timestamp = old_timestamp + timedelta(seconds=1)
                log[i][place]['time:timestamp'] = new_timestamp              
                for j in range(place+1, len(log[i])):
                    log[i][j]['time:timestamp'] += timedelta(seconds=2)
    return log

def add_one_swap(log, fraction):
    which = np.random.permutation(int(fraction*len(log)))
    for i in range(0, len(log)):
        if i in which:
            if len(log[i]) < 2:
                continue
            place_1 = random.randint(0, len(log[i])-1)
            place_2 = random.randint(0, len(log[i])-1)
            while place_1 == place_2:
                place_2 = random.randint(0, len(log[i])-1)
            event_1 = copy.deepcopy(log[i][place_1])
            event_2 = copy.deepcopy(log[i][place_2])
            log[i][place_1] = event_2
            log[i][place_2] = event_1
            if Has_Timestamp(log) == True:
                timestamp_1 = copy.deepcopy(log[i][place_1]['time:timestamp']) #switch timestamps to original place in trace
                timestamp_2 = copy.deepcopy(log[i][place_2]['time:timestamp'])
                log[i][place_1]['time:timestamp'] = timestamp_2
                log[i][place_2]['time:timestamp'] = timestamp_1
    return log


In [6]:
def create_log_no_noise(log_location, new_log_location):
    log = xes_importer.apply(log_location)
    #log = add_start_and_end(log, 'BOS', 'EOS')
    xes_exporter.apply(log, new_log_location)
    

In [16]:
def create_swap_logs(log_location, new_log_location, fractions):
    og_log = xes_importer.apply(log_location)
    #log = add_start_and_end(log, "BOS", "EOS")
    for fr in fractions:
        print(fr)
        log = copy.deepcopy(og_log)
        new_log = add_one_swap(log, fr)
        xes_exporter.apply(new_log, new_log_location+"_swap_"+"{:.2f}".format(fr)[2:]+".xes")
        
        
def create_delete_logs(log_location, new_log_location, fractions):
    og_log = xes_importer.apply(log_location)
    #log = add_start_and_end(log, "BOS", "EOS")
    for fr in fractions:
        print(fr)
        log = copy.deepcopy(og_log)
        new_log = add_one_delete(log, fr)
        xes_exporter.apply(new_log, new_log_location+"_delete_"+"{:.2f}".format(fr)[2:]+".xes")
        
def create_replace_logs(log_location, new_log_location, fractions):
    og_log = xes_importer.apply(log_location)
    #log = add_start_and_end(log, "BOS", "EOS")
    voc = get_voc(og_log)
    for fr in fractions:
        print(fr)
        log = copy.deepcopy(og_log)
        new_log = add_one_replace(log, fr, voc)
        xes_exporter.apply(new_log, new_log_location+"_replace_"+"{:.2f}".format(fr)[2:]+".xes")
        
def create_add_logs(log_location, new_log_location, fractions):
    og_log = xes_importer.apply(log_location)
    #log = add_start_and_end(log, "BOS", "EOS")
    voc = get_voc(og_log)
    for fr in fractions:
        print(fr)
        log = copy.deepcopy(og_log)
        new_log = add_one_addition(log, fr, voc)
        xes_exporter.apply(new_log, new_log_location+"_add_"+"{:.2f}".format(fr)[2:]+".xes")

In [18]:
fractions = [0.05, 0.10, 0.20, 0.40]

for choice in [0.0, 0.10, 0.20]:
    for parallel in [0.0, 0.10, 0.20]:
        for loop in [0.0, 0.10]:
            sequence = 1.0 - choice - parallel - loop
            print(sequence, choice, parallel, loop)
            name = "Choice"+"{:.2f}".format(choice)[2:]+"_Parallel"+"{:.2f}".format(parallel)[2:]+"_Loop"+"{:.2f}".format(loop)[2:]
            #create_log_no_noise("Experiment_2/Data/No_Noise/"+name+".xes", "Experiment2/Data/No_Noise/"+name+".xes")
            create_swap_logs("Experiment_2/Data/No_Noise/"+name+".xes", "Experiment_2/Data/Noise/"+name+"/Event_Log",fractions)
            create_delete_logs("Experiment_2/Data/No_Noise/"+name+".xes", "Experiment_2/Data/Noise/"+name+"/Event_Log",fractions)
            create_replace_logs("Experiment_2/Data/No_Noise/"+name+".xes", "Experiment_2/Data/Noise/"+name+"/Event_Log",fractions)
            create_add_logs("Experiment_2/Data/No_Noise/"+name+".xes", "Experiment_2/Data/Noise/"+name+"/Event_Log",fractions)

1.0 0.0 0.0 0.0


parsing log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]

0.05


exporting log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]

0.1


...

exporting log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]

0.2


exporting log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]

0.4


exporting log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]